In [16]:
using Pkg; Pkg.activate("."); Pkg.instantiate();Pkg.status()

  Activating project at `~/jcode/tutorial-tensornetwork/examples/qec`


Status `~/jcode/tutorial-tensornetwork/examples/qec/Project.toml`
  [0500ac79] TensorQEC v2.1.0
  [9a3f8284] Random v1.11.0


In [17]:
using TensorQEC
using TensorQEC.Yao
using TensorQEC.OMEinsum
using Random

## Circuit-level Quanutm Error Correction Decoding Problem
### Load data
The quantum circuits and the corresponding detector error model is placed under the `data` folder. Here we load the syndrome measurement circuit of `d=3` surface code and the corresponding detector error model.

In [ ]:
# Surface code
qc = parse_stim_file(joinpath(@__DIR__, "data", "surface_code_d=3_r=3.stim"), 26);
vizcircuit(qc)

In [18]:
dem = TensorQEC.parse_dem_file(joinpath(@__DIR__, "data", "surface_code_d=3_r=3.dem"))

┌─────────────┬────────────────┬──────────┐
│       Error │      Detectors │ Logicals │
├─────────────┼────────────────┼──────────┤
│  0.00193118 │            [1] │  Int64[] │
│  0.00193118 │         [1, 2] │  Int64[] │
│   0.0025292 │         [1, 9] │  Int64[] │
│  0.00193118 │         [2, 3] │  Int64[] │
│ 0.000400053 │      [2, 5, 6] │  Int64[] │
│ 0.000133387 │      [2, 5, 9] │  Int64[] │
│ 0.000799787 │         [2, 5] │     [25] │
│  0.00266191 │         [2, 6] │  Int64[] │
│ 0.000400053 │         [2, 9] │  Int64[] │
│  0.00411959 │            [2] │     [25] │
│  0.00405343 │            [3] │  Int64[] │
│  0.00219689 │         [3, 4] │  Int64[] │
│ 0.000533333 │      [3, 4, 7] │  Int64[] │
│  0.00059996 │         [3, 6] │  Int64[] │
│ 0.000200067 │      [3, 6, 7] │  Int64[] │
│ 0.000200067 │  [3, 6, 7, 10] │  Int64[] │
│      ⋮      │       ⋮        │    ⋮     │
└─────────────┴────────────────┴──────────┘
                           203 rows omitted


### Generate the tensor network
Now we can generate the corresponding tensor network with `compile` function. `TNMMAP` is a tensor network based marginal maximum a posteriori (MMAP) decoder, which finds the most probable logical sector after marginalizing out the error pattern on qubits. `TreeSA()` is the optimizer for optimizing the tensor network contraction order. `true` means that we want to factorize the tensors to rank-3 tensors to avoid large tensors.

In [6]:
compiled_decoder = compile(TNMMAP(TreeSA(), true), dem)
compiled_decoder.code

SlicedEinsum{Int64, DynamicNestedEinsum{Int64}}(Int64[], 244∘219∘793, 219∘793 -> 244
├─ 244∘219∘793
└─ 760∘219, 760∘793 -> 219∘793
   ├─ 243∘219∘760, 219∘243 -> 760∘219
   │  ├─ 243∘219∘760
   │  └─ 219, 243 -> 219∘243
   │     ├─ 219
   │     └─ 243
   └─ 760∘758∘190, 793∘190∘758 -> 760∘793
      ├─ 190∘218∘760, 758∘190∘218 -> 760∘758∘190
      │  ├─ 190∘218∘760
      │  └─ 758∘218, 218∘190 -> 758∘190∘218
      │     ├─ 759∘758, 218∘759 -> 758∘218
      │     │  ⋮
      │     │  
      │     └─ 218, 190 -> 218∘190
      │        ⋮
      │        
      └─ 793∘792∘753, 190∘792∘753∘758 -> 793∘190∘758
         ├─ 793∘216∘792, 753∘216 -> 793∘792∘753
         │  ├─ 793∘216∘792
         │  └─ 241∘216∘753, 241∘216 -> 753∘216
         │     ⋮
         │     
         └─ 752∘757∘190∘792, 753∘752∘758∘757 -> 190∘792∘753∘758
            ├─ 752∘757∘622∘790∘204, 190∘622∘790∘792∘204 -> 752∘757∘190∘792
            │  ⋮
            │  
            └─ 753∘215∘752, 758∘757∘215 -> 753∘752∘758∘757
       

In [19]:
contraction_complexity(compiled_decoder.code,uniformsize(compiled_decoder.code, 2))

Time complexity: 2^19.58676650225778
Space complexity: 2^11.0
Read-write complexity: 2^17.71708731655822

## Compute the probability of different logical sectors by tensor network contraction

In [20]:
# Randomly generate an error pattern and measure the syndrome.
Random.seed!(1234)
ep = random_error_qubits(IndependentFlipError(dem.error_rates))
syndrome = syndrome_extraction(ep, compiled_decoder.tanner)

# Update the syndrome and compute the probability of different logical sectors by tensor network contraction.
TensorQEC.update_syndrome!(compiled_decoder, syndrome)
mar = compiled_decoder.code(compiled_decoder.tensors...)

2-element Vector{Float64}:
 0.8426216150912977
 9.535387828248589e-8

In [ ]:
# Then we can use this probability to infer the error pattern.
_, pos = findmax(mar)
ep = TensorQEC._mixed_integer_programming_for_one_solution(compiled_decoder.tanner.H, syndrome.s)
if sum(x -> ep[x], compiled_decoder.l2q[1]).x == (pos == 1)
    ep[compiled_decoder.l2q[1]] .+= Mod2(1)
end

In [27]:
# Finally, we can check whether the error pattern is correct.
syndrome == syndrome_extraction(ep, compiled_decoder.tanner)

true

### Another hard example of [[144,12,12]] BB Code.
This file comes from https://github.com/quantumlib/tesseract-decoder/tree/main/testdata/bivariatebicyclecodes

In [ ]:
dem = TensorQEC.parse_dem_file(joinpath(@__DIR__, "data", "r=12,d=12,p=0.001,noise=si1000,c=bivariate_bicycle_X,nkd=[[144,12,12]],q=288,iscolored=True,A_poly=x^3+y+y^2,B_poly=y^3+x+x^2.dem"))
ct = compile(TNMMAP(TensorQEC.NoOptimizer(), true), dem); # Here we use the NoOptimizer to avoid any optimization. Since the code is too large, the default optimizer will be too slow.
length(ct.code.ixs)

585004

In [ ]:
contraction_complexity(ct.code,uniformsize(ct.code, 2))

Time complexity: 2^583276.0
Space complexity: 2^12.0
Read-write complexity: 2^22.024728467158457